In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from zipfile import ZipFile
from os import listdir
from os.path import isfile, join
import datetime
import time
import os
import numpy as np

In [3]:
path = "C:/Users/Mcian/Downloads/crypto_data/"

In [4]:
files = [f for f in listdir(path) if 'usd' not in f]

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(open('data/KuCoin trade volume and market listings _ CoinMarketCap.html'), 'html.parser')

In [9]:
coins = []
for t in soup.find_all("div", {"class": "sc-130rhjl-1 eIoiVE"}):
    if 'USDC' in t.text:
        continue
    coin = t.text.replace('/', '').replace('USDT', 'UST').replace('DOGE','dog').lower()+'.csv'
    coins.append(coin)

In [10]:
len(files), len(np.unique(files)), len(coins)

(133, 133, 93)

In [18]:
seq_len = 300
frame = '1T'

data = []
# old_file = file
# not_hit = True
for file in coins:
#     if not_hit and file != old_file:
#         continue
#     else:
#         not_hit = False
    if file not in files:
        print('missed', file)
        continue
        
    df = pd.read_csv(path+file)
    df = df[['time', 'close']]
    df['time'] = df['time'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    df = df.set_index('time')
    df = df[~df.index.duplicated()]
    if df.shape[0] == 0:
        continue
    print(file)
    df = df.resample(frame).fillna(None)

    temp = (~df.isna()).cumsum()
    df['index'] = np.concatenate((((temp[seq_len:].values - temp[:-seq_len].values) >= [0.9*seq_len])[:,0], [False]*seq_len))

    df = df.fillna(method='ffill')
    close_log = 100*50*(df.iloc[1:]['close'].values - df.iloc[:-1]['close'].values)/df.iloc[:-1]['close'].values
    close_log = np.concatenate(([None], np.sign(close_log)*np.log(abs(close_log)+1)))
    df['close_log'] = close_log
    df = df.iloc[1:]

    df = df.reset_index()
    idxs = df[df['index']].index
    prev_idxs = []
    count = 0
    for i in idxs:
        if i+2 in prev_idxs:
            continue
        seq = df.loc[i+2:i+seq_len]
        data.append(seq['close_log'].values)
        prev_idxs = list(range(i+2, i+seq_len))
        count += 1
    print(count*seq_len, df['index'].sum(), len(df), len(data))

btcust.csv
169800 137261 1166661 566
ethust.csv
134700 103276 1166663 1015
xrpust.csv
6300 3853 233623 1036
adaust.csv
4500 2094 426493 1051
dogust.csv
0 0 465476 1051
missed kcsust.csv
missed telust.csv
missed lunaust.csv
missed bnbust.csv
missed ustust.csv
ethbtc.csv
675600 595214 2746017 3303
ltcust.csv
3300 1050 1120575 3314
missed vetust.csv
dotust.csv
10200 6053 404351 3348
missed usdnust.csv
eosust.csv
1200 254 1120583 3352
missed maticust.csv
missed vraust.csv
xlmust.csv
0 0 165914 3352
missed qntust.csv
missed dashust.csv
missed nwcust.csv
missed linkust.csv
missed thetaust.csv
missed dagust.csv
missed atomust.csv
missed icpust.csv
missed shibust.csv
missed algoust.csv
missed chrust.csv
missed kokust.csv
missed karust.csv
missed bchust.csv
missed clvust.csv
missed nwcbtc.csv
missed trxust.csv
missed aaveust.csv
ksmust.csv
0 0 387687 3352
missed kcsbtc.csv
missed fearust.csv
missed compust.csv
missed enjust.csv
missed htrust.csv
missed lunaust.csv
uniust.csv
4500 2575 363235 33

In [19]:
data = np.array(data)

In [20]:
abs(data).mean(), data.min(), data.max()

(1.420354774357669, -6.577171619743256, 6.909542738913703)

In [40]:
for d in data:
    print(np.mean(d), np.min(d), np.max(d))

0.010805617070866282 -2.552613486649227 3.7106483988764456
0.028194650635908147 -1.912859389833042 2.3604574270273035
-0.03787215773336711 -2.8114463751232637 3.116413518868023
-0.027584100808717652 -3.4545192601538703 2.452358600047425
-0.01985302582402284 -2.1455808957391427 3.172896068570908
-0.06463067517470143 -2.6857262991858475 3.5663270538217575
0.01895866546913868 -1.7719178926743446 2.609753166340722
-0.026033922304224237 -2.334578068250021 1.9502394984450937
0.03710566849848856 -1.8988142968013473 2.4346265509239404
-0.019970836974007155 -1.9212662968943868 2.151620820536172
-0.06530999100124274 -2.5330115435454075 2.625564564567306
0.004082636618905162 -3.5298400687433458 3.603792895541823
-0.017494635022167884 -2.4739979975832362 2.6212761175492036
0.04573007230274369 -1.717102633834329 2.8858498722817534
-0.012782543857612384 -2.074045037340727 1.9311439964896284
-0.01709488416406415 -2.814604605407271 2.1075411478069905
-0.016935922198633234 -2.562470245535246 3.93378196

-0.005923261153563326 -4.3212833072044745 4.336054451589328
-0.007163312598202816 -4.283890851051176 3.9327509760927133
-0.0009584620254832697 -3.8992597061087 3.9087840054507255
0.04992433018806223 -4.322136524636115 4.2401901735508725
-0.014621914890284885 -3.950259853350956 3.9184388777823105
-0.0262992123214471 -3.9229035741194274 3.909030025180781
-0.028329838860940263 -3.8298057197007136 3.828214803287107
0.01827604788367557 -4.199217188553586 3.928156173154098
-0.023938351719379147 -4.582917414875626 4.592269873293355
-0.012314158591424477 -3.918685311229164 3.9330519045216836
-0.04950842517972399 -3.918685311229164 3.928405110838715
0.012229714651373581 -3.240634113785049 4.229723859571318
-0.022773621382876724 -3.889874960671167 4.233274038368123
-0.05215035678681361 -3.9330519045216836 4.297960011776352
0.0362966592821782 -3.2695389703790396 3.933731193726261
-0.0348369744619024 -4.3360127853869965 4.428378079216311
-0.002755527690581904 -3.8552304439783915 4.25268449121599
-

-0.15295752871158094 -4.1815025396212455 4.194282237558476
0.02568883197565127 -2.862295171627755 4.222011113213125
-0.09762536086095958 -4.418445507040766 4.077153169588249
-0.3347547852385807 -4.745793006841617 4.809958132399166
-0.06301266573201733 -4.600139021524418 4.781632040576217
-0.08721348773567923 -5.2175748727969244 5.280276675167918
-0.2512904738209371 -3.9256069282017294 3.6385603439242216
-0.2983171436033371 -4.611825729504297 4.730472595010141
-0.12851945629902423 -4.295677722544189 4.301911300360318
-0.18542154246360082 -3.5639053873911894 4.067508910355725
-0.022705714391178432 -4.5699719450833545 4.589067200077695
-0.07720894509059571 -4.504585960219611 4.522436222162168
-0.19704011008331493 -4.908045998854315 4.8867692338645545
-0.11575460661345699 -3.763776805891027 4.656967661051467
-0.2406759560672898 -4.848163611297129 4.771864725798907
-0.008112466672932965 -3.591819130488677 3.9501360870839908
-0.052996851624064575 -5.3789352919182045 4.492026776729145
0.02700

0.07835534808045092 -3.9054456599463423 3.6207295973251545
0.09108584872458557 -3.7729213959124284 3.391020063059877
-0.01324641890711394 -4.376978067935332 3.5780179775659797
0.018925390012226387 -3.5994391564116506 3.869650765389055
-0.07877972238088231 -3.2619665302306147 3.526376933984521
0.108600747084654 -3.011211648688768 3.6546720541803093
-0.010352381105990672 -3.3295134581186314 3.1927927114221584
0.032784645047142784 -3.6944351304636425 3.5380408602022655
0.08683090372983758 -4.0564295491804225 3.624469961920337
0.13157016395446625 -3.0118165757903843 2.814778419267714
0.1162246941883298 -3.4194957154640164 3.156656080387112
0.045656324782026296 -2.9089065264884484 3.094576075912021
-0.08132181752164153 -3.993328274579087 3.821699499632413
0.1354187837346723 -3.831917709185659 3.643090186197883
-0.023590368443799057 -3.524705289125901 3.423760936348783
-0.048988121168483285 -3.6720308465171185 3.80226468790745
0.0646824353326564 -3.6287954823309803 3.3885969146738435
0.06938

0.11474689616983812 -4.071126814933543 3.136255845462528
-0.06369657134104714 -4.351748424897619 3.0642453692941456
0.02086135080877171 -2.562330788252878 2.8575816029917673
0.03715794231159623 -3.4519519046641083 3.1579391679303996
-0.06085500002608498 -4.059889401306474 3.47806971980905
-0.07358493471670931 -3.7858658377923327 3.853435444579109
-0.026249477885391472 -3.745005581021526 2.9944922442652273
0.05608086908219709 -3.1010901727772064 2.591375843181204
-0.051871675217477785 -2.8428947093522803 2.8438062902911794
0.05222605168928477 -3.4212557660993457 3.796463694646749
0.12373780947163769 -3.974373173215369 4.175358436608189
-0.054187391265824496 -2.925444200247815 3.8550171490644334
-0.03815886222124848 -5.19454955508055 3.7247737868605624
-0.026564282713947972 -3.0077120019416035 3.8523479975239345
-0.08839424413778253 -3.0981483488619563 2.46833963231509
-0.06777323556543668 -4.541404167274239 3.7236601418753383
0.09120943912545718 -2.8557084990430464 3.4254172126948794
0.

...

0.10990400142474939 -2.978030462786298 3.7136469725102987
-0.09120791880636978 -3.736197730019358 3.0839931383707087
-0.012190601987304997 -3.2095534698281565 3.103686044065187
0.08718699410382037 -2.8448045568275586 3.138396483189168
0.03412290014353119 -2.572476492112538 2.794281787101611
-0.028300277189620127 -3.569291208515907 3.5754828506257557
-0.0540360186803039 -3.3432248794496524 2.940294943121487
0.08677030000891012 -2.6361818662001415 2.7377061355764307
0.03165596541684115 -3.075095840463317 3.011616394306032
0.04459027106289253 -3.709677601651604 2.7366429153788703
0.051971775761752305 -3.2343944736121473 2.8181001649197897
0.04422078298820116 -3.061429084210548 3.1353268399065453
0.06049910924827992 -3.0835689346072805 2.7225902652786718
-0.04279462005460852 -2.494406509983423 2.684945832756783
-0.0499654988633455 -3.3669311502838295 2.756202819194094
-0.04336648910462623 -2.6306454531144574 3.0891688809925686
0.05282699366785825 -2.8521246757974286 2.5532866017549085
-0.0

-0.017107179013242192 -1.9390841839848945 2.11506425412936
0.012382191010788536 -2.7447277926862097 3.094364534565612
0.03177007265722113 -1.7542797954888771 1.3795522047056785
0.00813230564197426 -2.1261753840073494 1.7003268970685115
0.018553109042017624 -2.488961743610064 2.1980434670453
0.02597987567070139 -2.5146474516417117 1.8982512774572868
0.020396207029139704 -2.371655682408354 3.153710495088485
0.002210767307461952 -3.3322045101752087 3.0510037018048495
0.09789503400341577 -4.063805858731591 4.525823841172472
0.0017561129929693625 -2.545961445216874 2.3873274892931433
-0.014125430305290397 -1.9147850042995247 1.5635238671663372
0.010351422894428376 -2.388343378487555 2.288801059173873
0.007573166772564913 -2.855460264229449 1.7229115708005525
0.03947277601802765 -2.670862718715728 2.316733369459073
0.011485310641889011 -1.7746645919379973 1.4109212296519176
0.004045757443093212 -2.035699754289711 2.0356761293556147
0.022101828542120724 -2.6853811626914963 2.037852666649979
0

0.002664925919279772 -3.038061162569337 3.1617910945847156
0.03554125774860272 -3.7589871970612583 3.5337659973824653
-0.0670470668043169 -4.217603383851239 3.936036519862739
-0.11646342699535937 -4.122541660127476 3.199005202493681
0.10445396187890069 -3.094656116005839 3.40067106820805
-0.04046653312195636 -3.1648341051557454 3.2902288053826836
0.027215634126797127 -4.012126532949153 3.8208977774840056
0.07331649976800333 -3.509373136701005 4.300083187490406
0.08134315339187802 -3.559812220864078 3.5139167753928624
-0.07297983860346015 -3.2954401093882435 3.649913689943148
0.03856287880720088 -2.8480786784071093 3.454101399995049
-0.06351124677870107 -4.110382185819379 3.3770446400761513
-0.02247843968827226 -3.746964435965793 3.264039329184775
-0.02618269911896294 -2.99036552316489 3.550360952339226
0.04645036530425274 -2.5847721937864376 2.9491836827205193
0.03998247625263393 -3.797838139749821 3.2432801749756917
0.07471147384054594 -3.2017975307068336 3.3903928271218375
0.01202178

-0.02787552514844333 -2.4735815004309636 2.4709192758172365
-0.028998939225867486 -2.208530283857101 2.3670416972772155
-0.006451329200401959 -2.0238560665418333 2.4193386735785176
-0.019881851057056747 -3.206396534699288 3.0249612975412608
0.021410693080116887 -2.2611998768311707 2.445207749669551
-0.12113249107701364 -2.9396972161806514 2.8318973275943677
-0.040517126301544616 -4.044538058225509 2.968256372405002
0.033367685355998065 -2.7869357033929756 2.9775855317135367
-0.010314655745508562 -2.403047267707836 2.789743882352576
0.04296834992876507 -2.103720543598072 2.219057621778628
0.01795504639205281 -1.864199221400621 2.4111310595875537
-0.05198491460844272 -3.400440591510747 2.849601654488771
-0.02392004190382432 -2.357826876748423 2.615124563403585
0.013973674652251336 -2.3448503018945313 2.456082409337466
0.04667678858390475 -2.517639310686021 2.6800017196083363
0.020275982393833866 -2.5294571808948505 1.9030346620358696
-0.020825830963918712 -4.212135095634288 5.10772998674

KeyboardInterrupt: 

In [26]:
df['time'].iloc[0], 1545730073

(1596710520000, 1545730073)

In [31]:
datetime.datetime.fromtimestamp(1545730073)

datetime.datetime(2018, 12, 25, 1, 27, 53)

In [28]:
df['time']/1000

0         1.596711e+09
1         1.596711e+09
2         1.596711e+09
3         1.596711e+09
4         1.596713e+09
              ...     
109648    1.622297e+09
109649    1.622298e+09
109650    1.622298e+09
109651    1.622298e+09
109652    1.622299e+09
Name: time, Length: 109653, dtype: float64

In [27]:
pd.to_datetime(df['time']/1000)

0        1970-01-01 00:00:01.596710520
1        1970-01-01 00:00:01.596710820
2        1970-01-01 00:00:01.596710940
3        1970-01-01 00:00:01.596711000
4        1970-01-01 00:00:01.596712740
                      ...             
109648   1970-01-01 00:00:01.622297400
109649   1970-01-01 00:00:01.622297760
109650   1970-01-01 00:00:01.622297940
109651   1970-01-01 00:00:01.622298240
109652   1970-01-01 00:00:01.622298660
Name: time, Length: 109653, dtype: datetime64[ns]

In [180]:
np.save('data/forex/all_forex_data_ffill_test.csv', data)

In [139]:
np.save('data/forex/all_usd_data2.csv', data)

In [143]:
np.save('data/forex/all_usd_data2_test.csv', data)

In [140]:
len(data)/3096720*2

0.14972551602986386